In [ ]:
from collections import Counter, defaultdict

docs = {
    "D1": (["X1","X2","X3"], "C1"),
    "D2": (["X1","X2","X4"], "C1"),
    "D3": (["X4","X5","X6"], "C2"),
}
D4 = ["X1","X4","X5"]

In [4]:
V = sorted({x for d, _ in docs.values() for x in d})
classes = sorted(set(c for _, c in docs.values()))

class_counts = Counter(c for _, c in docs.values())
N = len(docs)
priors = {c: class_counts[c]/N for c in classes}

In [ ]:
def multinomial_nb(doc, alpha=1):
    # Сколько всего токенов у класса, и частотность токенов у класса
    token_counts = {c: Counter() for c in classes}
    total_tokens = {c: 0 for c in classes}
    for words, cls in docs.values():
        token_counts[cls].update(words)
        total_tokens[cls] += len(words)

    # Подсчет вероятностей
    scores = {}
    for c in classes:
        # Априорная вероятность класса
        score = priors[c]
        # Перемножение с вероятностями увидеть токен при известном классе (частоты)
        denom = total_tokens[c] + alpha * len(V)
        for w in doc:
            num = token_counts[c][w] + alpha
            score *= num / denom
        scores[c] = score
    return scores

In [ ]:
def bernoulli_nb(doc, alpha=1):
    # Вычисление частот только уникальных в рамках каждого примера класса токенов
    df = {c: Counter() for c in classes}
    for words, cls in docs.values():
        for w in set(words):
            df[cls][w] += 1
    n_c = class_counts

    # Подсчет вероятностей
    scores = {}
    for c in classes:
        # Априорная вероятность класса
        score = priors[c]
        # Перемножение с вероятностями увидеть токен при известном классе (вхождение)
        denom = n_c[c] + 2*alpha
        for w in V:
            p = (df[c][w] + alpha) / denom
            # Обработка случая, когда токена нет в тексте
            if w in doc:
                score *= p
            else:
                score *= (1 - p)
        scores[c] = score
    return scores

In [7]:
print("Multinomial:", multinomial_nb(D4))
print("Bernoulli:", bernoulli_nb(D4))

Multinomial: {'C1': 0.0023148148148148147, 'C2': 0.0018289894833104707}
Bernoulli: {'C1': 0.005859375, 'C2': 0.007315957933241885}
